In [1]:
import numpy as np

from ppfft.tools.pad import pad, adj_pad
from ppfft.tools.new_fft import adj_new_ifft, adj_new_fft2
from ppfft.tools.frac_fft import adj_frac_fft
from ppfft.ppfft.ppfft import ppfft


# Adjoint of ``ppfft``

In this notebook we define the adjoint of the PPFFT operator. Since we have already defined the adjoint operator of each function used in ``ppfft``, we only need apply them one after the other.

The following function, which defines an inner product on complex matrices, will be used to check our adjoints.


In [2]:
def matrix_dot(a, b):
    return np.sum(np.conjugate(a) * b)


In [3]:
def adj_ppfft(hori_ppfft: np.ndarray, vert_ppfft: np.ndarray) -> np.ndarray:
    n, m = hori_ppfft.shape[0] - 1, hori_ppfft.shape[1]
    hori_aux = np.empty(shape=(n, m), dtype=complex)
    vert_aux = np.empty(shape=(n, m), dtype=complex)

    for k in range(m):
        q = hori_ppfft[::-1, k]
        q = pad(q, (m,))
        q = adj_frac_fft(q, 2 * (k - n), n)
        q = adj_pad(q, (n,))
        q = adj_new_ifft(q)
        hori_aux[:, k] = q

        q = vert_ppfft[::-1, k]
        q = pad(q, (m,))
        q = adj_frac_fft(q, 2 * (k - n), n)
        q = adj_pad(q, (n,))
        q = adj_new_ifft(q)
        vert_aux[:, k] = q

    hori_aux = adj_new_fft2(hori_aux)
    hori_a = adj_pad(hori_aux, (n, n))

    vert_aux = adj_new_fft2(vert_aux)
    vert_a = adj_pad(vert_aux, (n, n))
    vert_a = np.transpose(vert_a)

    return hori_a + vert_a


In [4]:
n = 10
a = np.random.rand(n, n)
b, c = np.random.rand(n+1, 2*n+1), np.random.rand(n+1, 2*n+1)
hori_ppfft, vert_ppfft = ppfft(a)
adj_b_c = adj_ppfft(b, c)


In [5]:
matrix_dot(a, adj_b_c) - (matrix_dot(hori_ppfft, b) +
                          matrix_dot(vert_ppfft, c))


(-1.871816903076251e-05+1.6782874290299787e-07j)